In [1]:
# NAO USAR A TABELA data_product, E SIM A store_product!!

# Remover primeiro os itens 'C/ 8', verificando se após o 'C/' a string comça com dígito.
# Após isso, remover diretamente os itens com 'C/', por exemplos 'C/8'
# Fazer o mesmo para os itens com a string 'S/' e 'REF'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('inputs/data_product_202208091047.csv', usecols = ['id', 'barcode', 'name'])

In [4]:
df.shape

(69111, 3)

In [5]:
df['name'].nunique()


48879

In [6]:
df.drop_duplicates(subset = 'name', inplace = True)

In [7]:
df.shape

(48879, 3)

In [8]:
df['name'] = df['name'].astype(str)

In [9]:
df['name'] = df['name'].str.upper()

In [10]:
df['cleaned'] = df['name']

In [11]:
def remove_pairs(name, element, condition):
    name = name.split(' ')
    cleaned = []

    for k, v in enumerate(name):
        if k + 1 < len(name):
            if condition == 'is_digit':
                if not(v == element and name[k + 1][0].isdigit()):
                    cleaned.append(v)

    return ' '.join(cleaned)

In [12]:
pairs = {
    'REF': 'is_digit',
    'R': 'is_digit'
}

In [13]:
for k, v in pairs.items():
    df['cleaned'] = df['cleaned'].apply(lambda x: remove_pairs(x, k, v))

In [14]:
# def remove_metrics(name):
#     to_remove = []
    
#     for word in name.split():
#         if word[0].isdigit() and 'X' in word and word[-1] == 'M' :
#             to_remove.append(word)

#     if len(to_remove) > 0:
#         return ''. join([name.replace(x, '') for x in to_remove])
    
#     return name

In [15]:
# df['cleaned'] = df['cleaned'].apply(lambda x: remove_metrics(x))

In [16]:
# def remove_startswith(name):
#     return lambda words: words.replace(''.join([x for x in words.split(' ') if x.startswith(name)]), '')

In [17]:
# startswith_ = ['(', 'REF', 'C/', 'S/', 'INTRAV']

In [18]:
# def remove_endswith(name, numbers):
#     return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(name) and x[:numbers].isdigit()]), '')

In [19]:
# endswith_ = {
#     'G':    -len('G'),
#     'L':    -len('L'),
#     'UN':   -len('UN'),
#     'ML':   -len('ML'),
#     'GR':   -len('GR'),
#     'CPS':  -len('CPS'),
#     'VOL':  -len('VOL'),
#     'CAP':  -len('CAP')
# }

In [20]:
# for name in startswith_:
#     df['cleaned'] = df['cleaned'].apply(remove_startswith(name))

In [21]:
# for k, v in endswith_.items():
#     df['cleaned'] = df['cleaned'].apply(remove_endswith(k, v))

In [22]:
df['cleaned'] = df['cleaned'].str.strip()

In [23]:
df['num_words'] = df['cleaned'].apply(lambda x: len(str(x).split(' ')))

In [24]:
items_cleaned = [
    52, 84
    # 48, 57, 58, 60, 76, 77, 82, 83, 84, 88, 94, 102, 106, 116, 118, 122, 171, 174
]

In [25]:
df[df['id'].isin(items_cleaned)].sort_values(by = 'id')

,id,barcode,name,cleaned,num_words
51,52,7897371801274,CHUP NUK R 729532,CHUP NUK,2
83,84,7898312327488,ESC PROART BAMBU REF 73252,ESC PROART,2


In [26]:
# df.sort_values(by = 'cleaned', inplace = True)

In [27]:
# df.to_csv('output.csv')

---

Tests

---

In [28]:
# df[df['id'].isin([52, 56, 63, 84, 169])]

In [29]:
# dict_ = {
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }